In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai.metrics import error_rate

torch.manual_seed(121)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(121)

In [ ]:
df = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
labels = np.array(df.iloc[:, 1:])
labels = np.where(labels==1)[1]

df['label'] = labels
df.to_csv('./train_data.csv', index=False)

In [ ]:
BS = 32
IMG_SZ = 512
CLASS_WT = torch.FloatTensor([1.0, 1.0, 1.0, 1.0])

GPU_IDS = [0,1,2]
torch.cuda.set_device(0)

label_dict = {0: 'healthy',
              1: 'multiple_diseases',
              2: 'rust',
              3: 'scab',
              'micro': 'micro',
              'macro': 'macro',
             }

In [ ]:
data_tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=None, max_zoom=1, max_lighting=None,
                          max_warp=0, p_affine=0, p_lighting=0)

data = ImageDataBunch.from_csv('../input/plant-pathology-2020-fgvc7', csv_labels='../../working/train_data.csv', folder='images', 
                               label_col=5, suffix='.jpg', valid_pct=0.2, ds_tfms=data_tfms,
                               bs=BS, size=IMG_SZ).normalize(imagenet_stats)
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.train_ds)
print(data.valid_ds)

In [ ]:
from sklearn.metrics import roc_curve, auc
class AUC(Callback):

    def __init__(self, num_cl, pick='micro'):
        self.id_to_class = label_dict
        self.name = str(self.id_to_class[pick])+'-AUC'
        self.pick = pick
        self.num_cl = num_cl
        
    
    def on_epoch_begin(self, **kwargs):
        self.outputs, self.targets = [], []
    
    def on_batch_end(self, last_output, last_target, **kwargs):
        self.outputs.append(last_output)
        self.targets.append(last_target)
    
    def on_epoch_end(self, last_metrics, **kwargs):
        self.outputs = F.softmax(torch.cat(self.outputs), dim=1).cpu().detach().numpy()
        self.targets = torch.cat(self.targets).cpu().numpy().reshape(-1)
        self.targets = np.eye(self.num_cl)[self.targets]
        fpr, tpr, roc_auc = {}, {}, {}
        for i in range(self.num_cl):
          fpr[i], tpr[i], _ = roc_curve(self.targets[:, i], self.outputs[:, i])
          roc_auc[self.id_to_class[i]] = auc(fpr[i], tpr[i])
        
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(self.targets.ravel(), self.outputs.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(self.num_cl)]))

        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(self.num_cl):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

        # Finally average it and compute AUC
        mean_tpr /= self.num_cl

        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        return add_metrics(last_metrics, roc_auc[self.id_to_class[self.pick]])


In [ ]:
# !pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

model_name = 'efficientnet-b7'
def getModel():
    model = EfficientNet.from_pretrained(model_name)
#     model._bn1 = nn.Identity()
    model._fc = nn.Linear(1536,data.c)
    return model

In [ ]:
learn = Learner(data, getModel(), model_dir='/kaggle/working/',
                    metrics=[accuracy, AUC(num_cl=4, pick=0), AUC(num_cl=4, pick=1), AUC(num_cl=4, pick=2), AUC(num_cl=4, pick=3), AUC(num_cl=4, pick='macro')])
# learn.loss_func = nn.CrossEntropyLoss(weight=CLASS_WT)
learn.model = torch.nn.DataParallel(learn.model, device_ids=GPU_IDS)

In [ ]:
# preds,y,losses = learn.get_preds(with_loss=True)
# interp = ClassificationInterpretation(learn, preds, y, losses)
# interp.plot_top_losses(9, figsize=(15,11))
# interp.plot_confusion_matrix(figsize=(6,6), dpi=60)

In [ ]:
learn.unfreeze()

learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-3)
learn.save('draft-efficientnet-B3')

In [ ]:
# learn.fit_one_cycle(2, max_lr=1e-3)
# learn.save('draft-efficientnet-B3')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=1e-4)
learn.save('draft-efficientnet-B3')
learn.recorder.plot_losses()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=1e-6)
learn.save('final-efficientnet-B3')

In [ ]:
test_df = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')

In [ ]:
test = ImageList.from_df(test_df, '../input/plant-pathology-2020-fgvc7/images/', suffix='.jpg')
data.add_test(test)
print(data)

In [ ]:
preds, *_ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
sub = pd.read_csv('../input/plant-pathology-2020-fgvc7/sample_submission.csv')
for lbl in range(4): 
    sub[[label_dict[lbl]]] = preds.detach().numpy()[:, lbl]
sub.head

In [ ]:
sub.to_csv('submission.csv', index=False)